## Speech Emotion Recognition

### Libraries :

In [ ]:
import os
import random
from IPython.display import Audio
import librosa
import librosa.display as disp
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearnex.model_selection import train_test_split

#import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.layers import Dense


ImportError: cannot import name 'ThreadpoolController' from 'threadpoolctl' (c:\ProgramData\Anaconda3\lib\site-packages\threadpoolctl.py)

In [2]:
! pip install threadpoolctl

Defaulting to user installation because normal site-packages is not writeable
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.5.1 requires joblib>=1.2.0, but you have joblib 1.1.0 which is incompatible.


In [1]:
path ='./speech_emotion/'

def load_data(path):
    f_emotions = []
    f_pathes = []
    folders = os.listdir(path)
    for folder in folders:
        files = os.listdir(path + folder)
        for file in files:
            step = file.split('.')[0]
            step = step.split('-')[2]
            f_emotions.append(int(step))
            f_pathes.append(path + folder + os.sep + file)
    return [f_emotions,f_pathes]
 
def get_emotion(number):
    
    info={1 : 'neutral',
          2 : 'calm' ,
          3 : 'happy',
          4 : 'sad',
          5 : 'angry',
          6 : 'fearful',
          7 : 'disgust',
          8 : 'surprised'}
    
    return info[number]
          
emotions, pathes = load_data(path)
        

NameError: name 'os' is not defined

In [5]:
def read_audio(path):
    data,sample_rate = librosa.load(path, duration=2.4, offset=0.6)
    
    return data, sample_rate

def draw_wave(path,data, sr):
    plt.figure(figsize=(10,4))
    #data, sr = librosa.load(path)
    plt.title('Audio Wave : ' + path, size =12)
    disp.waveshow(data,sr=sr)
    plt.show()

    
def draw_spectogram(path,data, sr):
    plt.figure(figsize=(10,4))
    #data, sr = librosa.load(path)
    X = librosa.stft(data)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.title('spectogram for Wave : ' + path, size =12 )
    disp.specshow(Xdb,sr=sr, x_axis ='time', y_axis = 'hz')
    plt.show()

    
def add_noise(data,sr):
    noise = 0.035 * np.random.uniform()*np.amax(data)
    data+= noise * np.random.normal(size=data.shape[0])
    
    return data, sr

def shift(data,sr):
    shift_range = int(np.random.uniform(low=-5, high=5)*1000)
    shifted = np.roll(data,shift_range)
    
    return shifted,sr

def pitch(data,sr,factor=0.7):
    pitched = librosa.effects.pitch_shift(y=data, sr = sr , n_steps=factor)
    
    return pitched, sr
    

def stretch(data,sr, rate=0.55):
    stretched = librosa.effects.time_stretch(y=data, rate=rate)
    
    return stretched, sr 

###  Extract Feature  MFCCs:

def feature_extraction(data,sr):
    mfcc = librosa.feature.mfcc(y=data, sr = sr)
    
    return mfcc
    
def processing_audio(data, sr, options):
    
    func = random.choice(options)
    
    if func == 'Standard':
        processed = data
    else:
        processed, _ = func(data,sr)
    return processed
    

In [6]:
def get_features(path):
    
    data,sample_rate = read_audio(path)
    
    funcs=['Standard',add_noise,pitch]
    
    features=[]
    
    func1_data = processing_audio(data, sample_rate, funcs)
    func2_data = processing_audio(func1_data, sample_rate, funcs)
    
    feature =  feature_extraction(func2_data,sample_rate)
    #print(feature.shape)
    
    if feature.shape == (20,104):
        features.append(feature)
    
    func1_data = processing_audio(data, sample_rate, funcs)
    func2_data = processing_audio(func1_data, sample_rate, funcs)
    feature =  feature_extraction(func2_data,sample_rate)
    
    if feature.shape == (20,104):
        features.append(feature)
    
    func1_data = processing_audio(data, sample_rate, funcs)
    func2_data = processing_audio(func1_data, sample_rate, funcs)
    feature =  feature_extraction(func2_data,sample_rate)
    
    if feature.shape == (20,104):
        features.append(feature)
    
    return features


def display(number):
    
    data, sample_rate = read_audio(pathes[number])
    mfcc_features = feature_extraction(data, sample_rate)
    print(len(mfcc_features))
    
    print(get_emotion(emotions[number]))
    
    #data, sample_rate = add_noise(data, sample_rate)
    #data, sample_rate = shift(data, sample_rate)
    #data, sample_rate = pitch(data, sample_rate)
    data, sample_rate = stretch(data, sample_rate)
    draw_wave(pathes[number], data, sample_rate)
    draw_spectogram(pathes[number], data, sample_rate)
    
    return data, sample_rate
    
d , sr = display(90)
Audio(data=d, rate=sr)

SystemError: initialization of _internal failed without raising an exception

In [ ]:
X = []
Y = []
for index in range(len(pathes)):
    value = get_features(pathes[index])
    if value != []:
        for item in value:
            X.append(item)
            Y.append(np.eye(8)[emotions[index] -1])#emotions

In [ ]:
#for index in range(len(pathes)):
 #   print (get_emotion(emotions[index]))
#1, n_classes =8
#np.eye(8)[index -1] #0->7

encoder = OneHotEncoder()
encoder.fit_transform(np.array([1,2,3,4,]).reshape(-1,1)).toarray()

In [ ]:
x_train , x_test , y_train ,y_test = train_test_split(np.array(X), np.array(Y),
                                                     test_size =0.2, random_state=11, shuffle=True)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
trainX = np.expand_dims(x_train, axis=3)
trainX = np.expand_dims(trainX, axis=3)
trainX = np.swapaxes(trainX, 1, 2)
print('trainX:' ,trainX.shape)


testX = np.expand_dims(x_test, axis=3)
testX  = np.expand_dims(testX , axis=3)
testX  = np.swapaxes(testX , 1, 2)
print('testX :' ,testX .shape)

inputShape = trainX.shape[1:]
inputShape

In [ ]:
#Create model 
def createModel(inputShape)